In [1]:
from moist_konrad import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

In [2]:
cases = [1.]
initial_temperatures = [240.,250.,260.,270.,275.,280.,285.,
                        288.,290.,292.5,295.,297.5,300.,302.5,305.,307.5,310.,315.]

initial_temperatures = [280.,285.,288.,290.,292.5,295.,297.5,300.,302.5,305.,307.5,310.,315.,320.]
initial_temperatures = [315.,320.]
#initial_temperatures = [280.,288.,307.5,310.]

depths = [20.]
solar_multiplier = [1.]
ini_rh = 0.9


land_atm_coupling = True
strong_coupling = True

albedo_sfc = 0.2

solar = 510
angle = 47.88

if strong_coupling == True:
    z_ref_T = 200
    z_ref_vmr = 2
else:
    z_ref_T = 2
    z_ref_vmr = 2

In [3]:
plev, phlev = konrad.utils.get_pressure_grids(1000e2, 100, 128)
atmosphere = konrad.atmosphere.Atmosphere(phlev)
clearsky = konrad.cloud.ClearSky.from_atmosphere(atmosphere)
surface = konrad.surface.SlabOcean(
                temperature=288,
                heat_sink=0,
                depth = 7.5,
                albedo = albedo_sfc)

radiation = konrad.radiation.RRTMG(solar_constant=solar, zenith_angle = angle)

temp_file_test = np.loadtxt('temp_file_strong-low-atm-temp-1.txt', delimiter=" ")
print(temp_file_test.shape)
T_atm_low_1 = temp_file_test[14,:]
atmosphere['T'][0] = T_atm_low_1
print(atmosphere['T'][0][0])
itop = coldpoint(atmosphere['T'][0])
print(np.min(atmosphere['T'][0]))
print(np.where(atmosphere['T'][0] == np.min(atmosphere['T'][0]))[0][0])
print(itop)

RH = manabe_rh(0.70, atmosphere['plev'])
atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],itop)
atmosphere['CO2'][0] = 1*348/1e6

radiation.update_heatingrates(atmosphere = atmosphere,surface = surface,cloud=clearsky)
    
rad_heat_atm = np.ones_like(radiation['net_htngrt'][0])

rad_heat_atm[:] = - np.diff(radiation['lw_flxu'][0] + radiation['sw_flxu'][0]-
                         (radiation['lw_flxd'][0] + radiation['sw_flxd'][0]))

atm_rad = np.sum(rad_heat_atm[:])
print(atm_rad)

RH = manabe_rh(0.70, atmosphere['plev'])
atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],itop)
atmosphere['CO2'][0] = 2*348/1e6

radiation.update_heatingrates(atmosphere = atmosphere,surface = surface,cloud=clearsky)
    
rad_heat_atm = np.ones_like(radiation['net_htngrt'][0])

rad_heat_atm[:] = - np.diff(radiation['lw_flxu'][0] + radiation['sw_flxu'][0]-
                         (radiation['lw_flxd'][0] + radiation['sw_flxd'][0]))

atm_rad = np.sum(rad_heat_atm[:])
print(atm_rad)

RH = manabe_rh(0.90, atmosphere['plev'])
atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],itop)
atmosphere['CO2'][0] = 1*348/1e6

radiation.update_heatingrates(atmosphere = atmosphere,surface = surface,cloud=clearsky)
    
rad_heat_atm = np.ones_like(radiation['net_htngrt'][0])

rad_heat_atm[:] = - np.diff(radiation['lw_flxu'][0] + radiation['sw_flxu'][0]-
                         (radiation['lw_flxd'][0] + radiation['sw_flxd'][0]))

atm_rad = np.sum(rad_heat_atm[:])
print(atm_rad)

RH = manabe_rh(0.90, atmosphere['plev'])
atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],itop)
atmosphere['CO2'][0] = 2*348/1e6

radiation.update_heatingrates(atmosphere = atmosphere,surface = surface,cloud=clearsky)
    
rad_heat_atm = np.ones_like(radiation['net_htngrt'][0])

rad_heat_atm[:] = - np.diff(radiation['lw_flxu'][0] + radiation['sw_flxu'][0]-
                         (radiation['lw_flxd'][0] + radiation['sw_flxd'][0]))

atm_rad = np.sum(rad_heat_atm[:])
print(atm_rad)

(36, 128)
287.4578909971114
202.26576919428683
57
57
-114.46976867982215
-113.73203536859654
-119.32953692235088
-118.17248198121985


In [4]:
results_file = open("results_file-rce-1.txt",'w')
lambda_file = open("lambda_file-rce-1.txt",'w')
temp_file = open("temp_file-rce-1.txt",'w')
water_file = open("water_file-rce-1.txt",'w')

In [ ]:
%%time

for k in range(len(solar_multiplier)):
    for de in range(len(depths)):
        for te in range(len(initial_temperatures)):
            print(depths[de],initial_temperatures[te],solar_multiplier[k])
            ###PARAMETERS OF THE TEST:

            #flux for efficiency computation:
            # 1: LH + SH
            # 2: Radiation surface
            # 3: Atmosphere radiation
            case = cases[0]
            depth_d = depths[de]


            #coupled surface-atmosphere: 
            #if true, surface fluxes and surface temperature change at each step, constant otherwise
            coupled = land_atm_coupling

            Ts_f = np.array([initial_temperatures[te]])
            print(Ts_f)

            RLS = 1/1

            #Strong coupling surface-atmosphere:
            #If true the atmosphere moist adiabat starts at surface temperature
            #If false find the atmopsheric moist adiabat that conserves energy
            strong = strong_coupling
            

            A = albedo_sfc

            ### INITIALIZATION
            E_imbalance = 0.

            plev, phlev = konrad.utils.get_pressure_grids(1000e2, 100, 128)
            atmosphere = konrad.atmosphere.Atmosphere(phlev)
            clearsky = konrad.cloud.ClearSky.from_atmosphere(atmosphere)
            surface = konrad.surface.SlabOcean(
                temperature=288,
                heat_sink=0,
                depth = depth_d,
                albedo = 0.5)

            radiation = konrad.radiation.RRTMG(solar_constant=solar, zenith_angle = angle)

            ndays = 10000
            timestep = 0.5 #days
            nsteps = int(ndays/timestep)

            printcounter = 0
            status = 0

            tol = 1e-6

            ### RCPE COMPUTATION

            surface['temperature'] = Ts_f
            surface.albedo = A
            atmosphere['T'][0] = moist_adiabat(surface['temperature'],atmosphere['T'][0],atmosphere)
            print(surface['temperature'],surface.albedo)

            conv_top = coldpoint(atmosphere['T'][0])
            RH = manabe_rh(ini_rh,atmosphere['plev'])
            atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],conv_top)
            T_atm_low = surface['temperature']

            print(surface['temperature'],surface.albedo)

            #### RCE FOR PRESENT-DAY CO2
            atmosphere['CO2'][0] = 348/1e6
            dLH = 0.
            Fconv = 0.

            T_sfc_1 = [1e8]
            toa_1 = [1e8]
            for i in range(nsteps):
                #surface fluxes
                z = height(atmosphere['plev'],atmosphere['T'][0],
                           atmosphere['phlev'][0],surface['temperature'])
                
                
                T_ini,vmr_ini = atmosphere['T'][0][0].copy(),atmosphere['H2O'][0,0].copy()

                #compute the fluxes or input the forcing
                
                LH = 0.9*Fconv
                SH = 0.1*Fconv


                #convective adjustment of the armosphere
                atmosphere,surface,radiation,net_rad_surface,atm_rad,\
                T_atm_low,Fconv,prec_mass,prec_heating,\
                RH,cold_point,prec_eff,water_content = RCE_step_DSE(timestep,
                                                              atmosphere,surface,radiation,clearsky,
                                                              SH,LH,A,T_atm_low,ini_rh)
                



                #update surface temperature with energetic fluxes (latent and radiative)
 
                surface['temperature'] = Ts_f


                status += 1
                if printcounter == 500:
                    print(status/2,surface['temperature'])
                    printcounter = 0
                printcounter += 1

                T_sfc_1.append(surface['temperature'])
                toa_1.append(radiation['toa'].copy())
                if abs(T_sfc_1[i]-T_sfc_1[i-1])<tol/10 and abs(toa_1[i]-toa_1[i-1])<tol/10:
                    print(i)
                    break

            P1 = prec_mass
            q1 = atmosphere['H2O'][0].copy()[0]
            RH1 = RH[0]
            
            heat_sink = radiation['toa'].copy()
            #heat_sink = net_rad_surface - Fconv
            print(heat_sink,Fconv,net_rad_surface,heat_sink+Fconv,net_rad_surface-heat_sink-Fconv)
            
            #### RCE FOR PRESENT-DAY CO2
            atmosphere['CO2'][0] *= 1
            print(atmosphere['CO2'][0][0])


            T_sfc_2 = [1e8]
            toa_2 = [1e8]
            olr_2 = [1e8]
            for i in range(nsteps):
                #surface fluxes
                z = height(atmosphere['plev'],atmosphere['T'][0],
                           atmosphere['phlev'][0],surface['temperature'])
                
                T_ini,vmr_ini = atmosphere['T'][0][0].copy(),atmosphere['H2O'][0,0].copy()

                #compute the fluxes or input the forcing
                LH = 0.9*Fconv
                SH = 0.1*Fconv


                #convective adjustment of the armosphere
                atmosphere,surface,radiation,net_rad_surface,atm_rad,\
                T_atm_low,Fconv,prec_mass,prec_heating,\
                RH,cold_point,prec_eff,water_content = RCE_step_DSE(timestep,
                                                              atmosphere,surface,radiation,clearsky,
                                                              SH,LH,A,T_atm_low,ini_rh)


                #update surface temperature with energetic fluxes (latent and radiative)
                surface['temperature'] = surface['temperature'] + ((net_rad_surface - Fconv - heat_sink)
                                           * seconds_day * timestep / surface.heat_capacity)



                status += 1
                if printcounter == 500:
                    print(status/2,surface['temperature'])
                    printcounter = 0
                printcounter += 1

                T_sfc_2.append(surface['temperature'])
                toa_2.append(radiation['toa'].copy())
                olr_2.append(radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy())

                if abs(T_sfc_2[i]-T_sfc_2[i-1])<tol and abs(toa_2[i]-toa_2[i-1])<tol:
                    print(i)
                    break
                    
            print('hehe')
            olr_rec = radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy()
            net_sw = radiation['sw_flxd'][0,-1].copy() - radiation['sw_flxu'][0,-1].copy()
            isr_rec = radiation['sw_flxd'][0,-1].copy()
            results_file.write(str(coupled)+" "+str(depth_d)+" "+str(case)+" "+str(Ts_f)
                               +" "+str(surface['temperature'])+" "+str(T_ini)
                               +" "+str(net_rad_surface)+" "+str(atm_rad)+" "+str(radiation['toa'])
                               +" "+str(olr_rec)+" "+str(net_sw)+" "+str(isr_rec)
                               +" "+str(Fconv)
                               +" "+str(LH)+" "+str(SH)+" "+str(prec_eff)
                               +" "+str(prec_mass)+" "+str(LH/Lv * seconds_day)
                               +" "+str(RH[0])+" "+str(water_content)
                               +" "+str(atmosphere['CO2'][0][0])
                               +"\n")
            
            if de == 0:
                np.savetxt(temp_file,atmosphere['T'][0].reshape(1, -1).copy())
                np.savetxt(water_file,atmosphere['H2O'][0].reshape(1, -1).copy())
            
            heat_sink = radiation['toa'].copy()
            #heat_sink = net_rad_surface - Fconv
            print(heat_sink,Fconv,net_rad_surface,heat_sink+Fconv,net_rad_surface-heat_sink-Fconv)
                    
            #### RCE FOR PRESENT-DAY CO2
            atmosphere['CO2'][0] *= 2
            print(atmosphere['CO2'][0][0])


            T_sfc_2 = [1e8]
            toa_2 = [1e8]
            olr_2 = [1e8]
            for i in range(nsteps):
                #surface fluxes
                z = height(atmosphere['plev'],atmosphere['T'][0],
                           atmosphere['phlev'][0],surface['temperature'])
                
                T_ini,vmr_ini = atmosphere['T'][0][0].copy(),atmosphere['H2O'][0,0].copy()

                #compute the fluxes or input the forcing
                LH = 0.9*Fconv
                SH = 0.1*Fconv


                #convective adjustment of the armosphere
                atmosphere,surface,radiation,net_rad_surface,atm_rad,\
                T_atm_low,Fconv,prec_mass,prec_heating,\
                RH,cold_point,prec_eff,water_content = RCE_step_DSE(timestep,
                                                              atmosphere,surface,radiation,clearsky,
                                                              SH,LH,A,T_atm_low,ini_rh)


                #update surface temperature with energetic fluxes (latent and radiative)
                surface['temperature'] = surface['temperature'] + ((net_rad_surface - Fconv - heat_sink)
                                           * seconds_day * timestep / surface.heat_capacity)



                status += 1
                if printcounter == 500:
                    print(status/2,surface['temperature'])
                    printcounter = 0
                printcounter += 1

                T_sfc_2.append(surface['temperature'])
                toa_2.append(radiation['toa'].copy())
                olr_2.append(radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy())

                if abs(T_sfc_2[i]-T_sfc_2[i-1])<tol*10 and abs(toa_2[i]-toa_2[i-1])<tol*10:
                    print(i)
                    break

            print('hehe2')
            olr_rec = radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy()
            net_sw = radiation['sw_flxd'][0,-1].copy() - radiation['sw_flxu'][0,-1].copy()
            isr_rec = radiation['sw_flxd'][0,-1].copy()
            results_file.write(str(coupled)+" "+str(depth_d)+" "+str(case)+" "+str(Ts_f)
                               +" "+str(surface['temperature'])+" "+str(T_ini)
                               +" "+str(net_rad_surface)+" "+str(atm_rad)+" "+str(radiation['toa'])
                               +" "+str(olr_rec)+" "+str(net_sw)+" "+str(isr_rec)
                               +" "+str(Fconv)
                               +" "+str(LH)+" "+str(SH)+" "+str(prec_eff)
                               +" "+str(prec_mass)+" "+str(LH/Lv * seconds_day)
                               +" "+str(RH[0])+" "+str(water_content)
                               +" "+str(atmosphere['CO2'][0][0])
                               +"\n")
            
            if de == 0:
                np.savetxt(temp_file,atmosphere['T'][0].reshape(1, -1).copy())
                np.savetxt(water_file,atmosphere['H2O'][0].reshape(1, -1).copy())

            P2 = prec_mass
            q2 = atmosphere['H2O'][0].copy()[0]
            RH2 = RH[0]

            T1 = T_sfc_1[-1]
            T2 = np.array(T_sfc_2[1:])
            toa2 = np.array(toa_2[1:])
            olr2 = np.array(olr_2[1:])


            T_anom = T2 - T1

            lambda_1 = linregress( T_anom[200:,0], toa2[200:,0]).slope
            lambda_2 = linregress( T_anom[200:,0], olr2[200:]).slope


            lambda_file.write(str(coupled)+" "+str(depth_d)+" "+str(case)+" "+str(Ts_f)
                             +" "+str(olr_rec)+" "+str(net_sw)+" "+str(isr_rec)
                             +" "+str(T_anom[-1])+" "+str(lambda_1)+" "+str(lambda_2)
                             +" "+str((P2-P1)/P1*100/T_anom[-1])
                             +" "+str((q2-q1)/q1*100/T_anom[-1])
                             +" "+str(np.max(toa2))+" "+str(np.min(olr2))
                             +"\n")
    


20.0 315.0 1.0
[315.]
[315.] 0.2
[315.] 0.2
250.5 [315.]
500.5 [315.]
750.5 [315.]
1000.5 [315.]
1250.5 [315.]
1500.5 [315.]
1750.5 [315.]
2000.5 [315.]
2250.5 [315.]
2500.5 [315.]
2750.5 [315.]
3000.5 [315.]
3250.5 [315.]
3500.5 [315.]
3750.5 [315.]
4000.5 [315.]
4250.5 [315.]
4500.5 [315.]
4750.5 [315.]
5000.5 [315.]
5250.5 [315.]
5500.5 [315.]
5750.5 [315.]
6000.5 [315.]
6250.5 [315.]
6500.5 [315.]
6750.5 [315.]
7000.5 [315.]
7250.5 [315.]
7500.5 [315.]
7750.5 [315.]
8000.5 [315.]
8250.5 [315.]
8500.5 [315.]
8750.5 [315.]
9000.5 [315.]
9250.5 [315.]
9500.5 [315.]
9750.5 [315.]
[-54.63424824] 230.15718130373293 175.5274523863826 [175.52293306] [0.00451932]
0.000348
10000.5 [314.99996589]
10250.5 [314.99978666]
10500.5 [314.99970985]
10750.5 [314.99964343]
11000.5 [314.99962158]
11250.5 [314.99958853]
11500.5 [314.99957182]
11750.5 [314.99953208]
12000.5 [314.9995392]
12250.5 [314.99954439]
12500.5 [314.99955187]
12750.5 [314.99952378]
13000.5 [314.99953574]
13250.5 [314.99954304]
135

In [ ]:
temp_file.close()
water_file.close()
results_file.close()
lambda_file.close()

In [ ]:
print(T_anom[200:,0])

In [ ]:
plt.scatter(T_anom[:,0], olr2[:], s = 2)
plt.show()
plt.scatter(T_anom[:,0], toa2[:,0], s = 5)
plt.show()

In [ ]:
lambda_1 = linregress( T_anom[500:,0], toa2[500:,0]).slope
lambda_2 = linregress( T_anom[500:,0], olr2[500:]).slope
print(lambda_1,lambda_2)

In [ ]:
print(radiation['sw_flxd'][0,-1])

In [ ]:
print(plev)
print(phlev)
print(z)